Description: This program creates meteogams for select locations within the analysis domain. Surface data were obtained from the Iowa Environmental Mesonet at:
        https://mesonet.agron.iastate.edu/ASOS/
Portions (most) of this code were adapted from the MetPy meteogram example located at:
        https://unidata.github.io/MetPy/latest/examples/meteogram_metpy.html

In [2]:
import warnings
warnings.simplefilter('ignore')
import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from metpy.plots import add_metpy_logo, current_weather, sky_cover, StationPlot, wx_code_map
from metpy.units import units
import numpy as np
# Parse dates from .csv file, knowing their format as a string and convert to datetime
def parse_date(date):
    return dt.datetime.strptime(date, '%Y-%m-%d %H:%M')
#Process station data
NDf='data/SFC/ND_asos_022419.txt'

NDdata = pd.read_csv(NDf, header=0,
                       names=['stid', 'time', 'lon', 'lat', 'air_temperature', 'dew_point_temperature',
                              'wind_dir', 'wind_speed', 'slp', 'visibility', 
                              'wind_gust', 'weather', 'cloud_fraction'],
                       na_values='M')
MNf='data/SFC/MN_asos_022419.txt'
MNdata = pd.read_csv(MNf, header=0,
                       names=['stid', 'time', 'lon', 'lat', 'air_temperature', 'dew_point_temperature',
                              'wind_dir', 'wind_speed', 'slp', 'visibility', 
                              'wind_gust', 'weather', 'cloud_fraction'],
                       na_values='M')
#Define data for plotting. 
#datast = NDdata[NDdata['stid'] == 'GFK']
data1 = NDdata[NDdata['stid'] == 'GFK']
data2 = MNdata[MNdata['stid'] == 'TVF']
data3 = MNdata[MNdata['stid'] == 'BBB']

# Make meteogram plot
class Meteogram(object):
    """ Plot a time series of meteorological data from a particular station as a
    meteogram with standard variables to visualize, including thermodynamic,
    kinematic, and pressure. The functions below control the plotting of each
    variable.
    TO DO: Make the subplot creation dynamic so the number of rows is not
    static as it is currently. """

    def __init__(self, fig, dates, probeid, time=None, axis=0):
        """
        Required input:
            fig: figure object
            dates: array of dates corresponding to the data
            probeid: ID of the station
        Optional Input:
            time: Time the data is to be plotted
            axis: number that controls the new axis to be plotted (FOR FUTURE)
        """
        if not time:
            time = dt.datetime.utcnow()
        self.start = dates[0]
        self.fig = fig
        self.end = dates[-1]
        self.axis_num = 0
        self.dates = mpl.dates.date2num(dates)
        self.time = time.strftime('%Y-%m-%d %H:%M UTC')
        self.title = 'Latest Ob Time: {0}\nProbe ID: {1}'.format(self.time, probeid)

    def plot_winds(self, name, panel, ws, wd, wsmax, letter, plot_range=None):
        """
        Required input:
            ws: Wind speeds (knots)
            wd: Wind direction (degrees)
            wsmax: Wind gust (knots)
        Optional Input:
            plot_range: Data range for making figure (list of (min,max,step))
        """
        # PLOT WIND SPEED AND WIND DIRECTION
        self.ax1 = fig.add_subplot(3, 1, panel)
        ln1 = self.ax1.plot(self.dates, ws, label='Sustained',color="gray",linewidth=3)
        self.ax1.fill_between(self.dates, ws, 0,color="gray")
        self.ax1.set_xlim(self.start, self.end)
        if not plot_range:
            plot_range = [10, 50, 5]
        self.ax1.set_ylabel('Wind Speed (kts)', multialignment='center',size='xx-large')
        self.ax1.set_ylim(plot_range[0], plot_range[1], plot_range[2])
        self.ax1.grid(b=True, which='major', axis='y', color='b', linestyle='--',
                      linewidth=1)
        ln2 = self.ax1.plot(self.dates, wsmax, '.r', label='Gusts',linewidth=3,markersize=16)

        ax7 = self.ax1.twinx()
        ln3 = ax7.plot(self.dates, wd, linewidth=3, label='Visibility',color="black")
        ax7.set_ylabel('Visibility\n(mi)', multialignment='center',size='xx-large')
        ax7.set_ylim(0, 4)
        #ax7.set_yticks(np.arange(45, 405, 90), ['NE', 'SE', 'SW', 'NW'])
        lns = ln1 + ln2 + ln3
        labs = [l.get_label() for l in lns]
        ax7.xaxis.set_major_formatter(mpl.dates.DateFormatter('%H UTC'))
        ax7.set_title(name+' - 24 Feb. 2019',fontweight="bold",fontsize=16)
        ax7.text(self.start, 4.3, letter, fontsize=16,weight='bold',color="black")
        if panel == 1:
            ax7.legend(lns, labs, loc='upper center',bbox_to_anchor=(0.17, 1.02), ncol=3, prop={'size': 14})
#convert pandas time to datetime format
times1=[dt.datetime.strptime(x, '%Y-%m-%d %H:%M') for x in data1['time']]
times2=[dt.datetime.strptime(x, '%Y-%m-%d %H:%M') for x in data2['time']]
times3=[dt.datetime.strptime(x, '%Y-%m-%d %H:%M') for x in data3['time']]
# set the starttime and endtime for plotting, 24 hour range
#endtime = dt.datetime(2019, 2, 25, 0, 0, 0, 0)
#starttime = endtime - dt.timedelta(hours=24)
#settings for plot
mpl.rcParams['axes.linewidth'] = 2 #set the value globally
mpl.rcParams['xtick.labelsize'] = 16 #set the value globally
mpl.rcParams['ytick.labelsize'] = 16 #set the value globally

fig = plt.figure(figsize=(20, 10),dpi=300)

meteogram = Meteogram(fig, times1,'KGFK')
meteogram.plot_winds('Grand Forks, ND - KGFK',1,data1['wind_speed'], data1['visibility'], data1['wind_gust'],'(a)')
meteogram = Meteogram(fig, times2,'KTVF')
meteogram.plot_winds('Thief River Falls, MN - KTVF',2,data2['wind_speed'], data2['visibility'], data2['wind_gust'],'(b)')
meteogram = Meteogram(fig, times3,'KBBB')
meteogram.plot_winds('Benson, MN - KBBB',3,data3['wind_speed'], data3['visibility'], data3['wind_gust'],'(c)')
#meteogram.plot_thermo(tempc,dpc )
fig.subplots_adjust(hspace=0.5, wspace=0.5)
plt.savefig('KJ_MWR_2020_Fig04_final.png', dpi=300, bbox_inches='tight')
plt.savefig('KJ_MWR_2020_Fig04_final.pdf',format='pdf', bbox_inches='tight')
plt.show()

<Figure size 6000x3000 with 6 Axes>